In [0]:
#@title
# Returns a list of Course objects named courses
def class_search(course_codes, courses, session, exclude_quiz, registrations = ""):
    from time import sleep
    from json.decoder import JSONDecodeError
    # Dictionary from department names to list of course codes within that department
    course_dict = dict()

    for code in course_codes:
        # Removes beginning and trailing whitespace
        code = code.strip()

        # Extracts solely the department name from the course code provided by the user
        # Ex: ACCT from ACCT-410
        dept = code[:code.find("-")]

        if dept in course_dict.keys():
            course_dict[dept].append(code)
        else:
            course_dict[dept] = [code]

    # Dictionary from dept to response for that department
    responses = dict()

    for key in course_dict:
        # Request header
        semester = "20201"
        url = "https://web-app.usc.edu/web/soc/api/classes/" + key + "/" + semester

        dept_res = None
        i = 0
        while (dept_res == None):
            try:
                dept_res = session.get(url, timeout = 3).json()
                i+=1
                # Exits if could not retrieve data after 5 iterations
                if i > 4:
                    sys.exit("Could not retrieve course data from API, please restart program.")
                sleep(1)
            except JSONDecodeError:
                sys.exit(key + " is not a valid department")

        responses[key] = dept_res

        for code in course_dict[key]:
            try:
                course = tuple(filter(lambda x: x["PublishedCourseID"] == code, responses[key]["OfferedCourses"]["course"]))[0]
            except IndexError:
                year = semester[:4]
                season = ""
                if semester[-1] == '1':
                    season = "Spring" 
                elif semester[-1] == '2':
                    season = "Summer" 
                elif semester[-1] == '3':
                    season = "Fall"
                sys.exit(code + " is not a valid course for " + season + " " + year)

            sec_data = course["CourseData"]["SectionData"]

            course = Course(code)
            # If there are multiple sections
            if isinstance(sec_data, list):
                for section in sec_data:
                    if not ((section["type"] == "Qz") and (exclude_quiz == "Y")):
                        section_id = section["id"]
                        open_spaces = (int(section["spaces_available"]) - int(section["number_registered"]))
                        # If a given section has any open spaces or if it is in the set of sections the user is already registered for
                        if (open_spaces > 0) or (section_id in registrations):
                            # If the section has an instructor
                            if "instructor" in section.keys():
                                # If there is only one instructor
                                if isinstance(section["instructor"], dict):
                                    course.add_section(Section(code, section["id"], section["type"], section["day"], section["start_time"], section["end_time"], section["location"], (section["instructor"]["first_name"] + " " + section["instructor"]["last_name"])))
                                # If there are multiple instructors
                                else:
                                    instructors = []
                                    for instructor in section["instructor"]:
                                        instructors.append(instructor["first_name"] + " " + instructor["last_name"])
                                    course.add_section(Section(code, section["id"], section["type"], section["day"], section["start_time"], section["end_time"], section["location"], set(instructors)))
                            else:
                                course.add_section(Section(code, section["id"], section["type"], section["day"], section["start_time"], section["end_time"], section["location"]))
            # If there is only one section
            else:
                section_id = sec_data["id"]
                open_spaces = (int(sec_data["spaces_available"]) - int(sec_data["number_registered"]))
                if (open_spaces > 0) or (section_id in registrations):
                    if "instructor" in sec_data.keys():
                        if isinstance(sec_data["instructor"], dict):
                            course.add_section(Section(code, sec_data["id"], sec_data["type"], sec_data["day"], sec_data["start_time"], sec_data["end_time"], sec_data["location"], (sec_data["instructor"]["first_name"] + " " + sec_data["instructor"]["last_name"])))
                        else:
                            instructors = []
                            for instructor in sec_data["instructor"]:
                                instructors.append(instructor["first_name"] + " " + instructor["last_name"])
                            course.add_section(Section(code, sec_data["id"], sec_data["type"], sec_data["day"], sec_data["start_time"], sec_data["end_time"], sec_data["location"], set(instructors)))
                    else:
                        course.add_section(Section(code, sec_data["id"], sec_data["type"], sec_data["day"], sec_data["start_time"], sec_data["end_time"], sec_data["location"]))

            # If there are no available sections for the course
            if not course.sections:
                print(code, "has no available sections")
                exit()

            courses.append(course)

    return courses

In [0]:
#@title
from datetime import datetime as dt
import sys

class Section:
    def __init__(self, course_code, section_id, section_type, day, start_time, end_time, location, instructor = ""):
        self.code = course_code
        self.id = section_id
        self.type = section_type
        self.days = day
        self.start = self.getTime(start_time, section_id, course_code)
        self.end = self.getTime(end_time, section_id, course_code)
        self.location = self.building(location, section_type)
        self.professor = instructor

    @staticmethod
    def getTime(time, section_id, course_code, format = '%H:%M'):
        try:
            return dt.strptime(time, format)
        except ValueError:
            sys.exit("Section " + section_id + " from course " + course_code + " has a TBA time")

    def building(self, loc, section_type):
        try:
            if (loc == 'TBA') and (section_type != 'Qz'):
                raise AttributeError
            # Dictionary of USC building abbreviations to their coordinates, generated through the Google Maps API
            directory = {'ABA': {'Address': 'Anna Bing Arnold Child Care Ctr., 2715 University Ave.', 'Coordinates': {'lat': 34.0717132, 'lng': 241.8310832}}, 'ABM': {'Address': 'American Button Manufacturing, 2823 S. Flower St.', 'Coordinates': {'lat': 34.0254841, 'lng': 241.723581}}, 'ABX': {'Address': 'Anna Bing Arnold Child Care Ctr. Annex, 2716 Severance St., 90007', 'Coordinates': {'lat': 34.0717132, 'lng': 241.8310832}}, 'ACB': {'Address': 'Ahmanson Ctr. for Biological Research, 825 Bloom Walk', 'Coordinates': {'lat': 34.0192383, 'lng': 241.7130538}}, 'ACC': {'Address': 'Accounting, Leventhal School of, 3660 Trousdale Pkwy.', 'Coordinates': {'lat': 34.0191839, 'lng': 241.71445640000002}}, 'ACX': {'Address': 'Ahmanson Ctr. for Biological Research Animal Section, 825 Bloom Walk', 'Coordinates': {'lat': 34.0582486, 'lng': 241.7526997}}, 'ADM': {'Address': 'George Finley Bovard Administration Bldg. and Kenneth Norris, Jr. Aud., 3551 Trousdale Pkwy.', 'Coordinates': {'lat': 34.0209141, 'lng': 241.7144629}}, 'ADX': {'Address': 'Alpha Delta Chi House, 725-731 W. 32nd St.', 'Coordinates': {'lat': 37.2049111, 'lng': 266.72209499999997}}, 'AEP': {'Address': 'Alpha Epsilon Pi fraternity, 904 W. 28th St.', 'Coordinates': {'lat': 34.028203, 'lng': 241.71739530000002}}, 'AES': {'Address': 'Alice and Eleonore Schoenfeld Symphonic Hall, 3450 Watt Way', 'Coordinates': {'lat': 34.0234263, 'lng': 241.7138423}}, 'AHF': {'Address': 'Allan Hancock Foundation Bldg., 3616 Trousdale Pkwy.', 'Coordinates': {'lat': 34.019763, 'lng': 241.714751}}, 'ALM': {'Address': 'Alumni House, Widney, 635 Childs Way', 'Coordinates': {'lat': 34.0194996, 'lng': 241.7171616}}, 'ANH': {'Address': 'Annenberg House, 711 W. 27th St.', 'Coordinates': {'lat': 34.0287918, 'lng': 241.7207981}}, 'ANN': {'Address': 'Wallis Annenberg Hall, 3630 Watt Way', 'Coordinates': {'lat': 34.0208559, 'lng': 241.7128964}}, 'ASC': {'Address': 'Annenberg School for Communication and Journalism, 3502 Watt Way', 'Coordinates': {'lat': 34.02211450000001, 'lng': 241.7138728}}, 'ATM': {'Address': 'At Mateo, 1262 Palmetto St., Ste. 515', 'Coordinates': {'lat': 34.0396448, 'lng': 241.7667635}}, 'BAA': {'Address': 'Bel-Aire Apts., 1124 W. 29th St.', 'Coordinates': {'lat': 40.7538737, 'lng': 285.9927503}}, 'BAB': {'Address': 'Barracks A & B, 1200 N. State St.', 'Coordinates': {'lat': 34.0577573, 'lng': 241.7918383}}, 'BAR': {'Address': 'Barlow Hospital, 2000 Stadium Way', 'Coordinates': {'lat': 34.0757094, 'lng': 241.7518147}}, 'BCC': {'Address': 'Eli and Edythe Broad CIRM Center for Regenerative Medicine and Stem Cell Research, 1425 San Pablo St.', 'Coordinates': {'lat': 34.0619293, 'lng': 241.79633230000002}}, 'BCH': {'Address': 'Boone Center Housing, 10317-10343 W. Big Fishermans Cove, Catalina Island', 'Coordinates': {'lat': 33.3878856, 'lng': 241.58368969999998}}, 'BDF': {'Address': 'Brooks Memorial Pavilion and Dedeaux Field, 1021 Childs Way', 'Coordinates': {'lat': 34.0237631, 'lng': 241.71014780000002}}, 'BHE': {'Address': 'Biegler Hall of Engineering, 920 Downey Way', 'Coordinates': {'lat': 34.0205484, 'lng': 241.7114862}}, 'BIM': {'Address': 'Brittingham Intramural Field, 3506 McClintock Ave.', 'Coordinates': {'lat': 34.022755, 'lng': 241.7121656}}, 'BIT': {'Address': 'Bing Theatre, 3500 Watt Way', 'Coordinates': {'lat': 34.0224359, 'lng': 241.71394279999998}}, 'BKF': {'Address': 'Brian Kennedy Field, 3515 McClintock Ave.', 'Coordinates': {'lat': 34.0230348, 'lng': 241.7110252}}, 'BKS': {'Address': 'Pertusati University Bookstore, 840 Childs Way', 'Coordinates': {'lat': 34.0206239, 'lng': 241.7134639}}, 'BMH': {'Address': 'Booth Ferris Rehearsal Hall, 820 W. 34th St.', 'Coordinates': {'lat': 40.7527048, 'lng': 286.0053297}}, 'BMP': {'Address': 'Burbank Medical Plaza #2, 191 S. Buena Vista St., #310, Burbank', 'Coordinates': {'lat': 34.1584843, 'lng': 241.67089679999998}}, 'BMT': {'Address': 'Bishop Medical Teaching and Research Bldg., 1333 San Pablo St.', 'Coordinates': {'lat': 34.0600391, 'lng': 241.7952393}}, 'BRI': {'Address': 'Barrack Hall, 3670 Trousdale Pkwy.', 'Coordinates': {'lat': 34.0188316, 'lng': 241.7142619}}, 'BSR': {'Address': 'Birnkrant Residential College, 642 W. 34th St.', 'Coordinates': {'lat': 34.0215193, 'lng': 241.7177055}}, 'BTP': {'Address': 'Beta Theta Pi fraternity, 2714 Portland St.', 'Coordinates': {'lat': 34.0293439, 'lng': 241.7172354}}, 'CAB': {'Address': 'CMSC Administration Bldg., P.O. Box 398, Catalina Island, 90704', 'Coordinates': {'lat': 33.3878856, 'lng': 241.58368969999998}}, 'CAL': {'Address': 'Carole Little Bldg., 3434 S. Grand Ave.', 'Coordinates': {'lat': 34.0191601, 'lng': 241.7234911}}, 'CAP': {'Address': 'Century Apts., 3115 S. Orchard Ave.', 'Coordinates': {'lat': 34.0262676, 'lng': 241.71041789999998}}, 'CAR': {'Address': 'Cardinal Gardens, 3131 S. McClintock Ave.', 'Coordinates': {'lat': 34.0259371, 'lng': 241.7126449}}, 'CAS': {'Address': 'College Academic Services Bldg., 3454 Trousdale Pkwy.', 'Coordinates': {'lat': 34.0223665, 'lng': 241.71622200000002}}, 'CBB': {'Address': 'Century Boulevard Building, 6033 Century Blvd. #920, Los Angeles', 'Coordinates': {'lat': 33.9454984, 'lng': 241.7083044}}, 'CBG': {'Address': 'University Gardens Apts., 1250 W. Jefferson Blvd.', 'Coordinates': {'lat': 34.0247839, 'lng': 241.70731999999998}}, 'CCC': {'Address': 'Health Sciences Child Care Ctr., 2250 Alcazar St.', 'Coordinates': {'lat': 34.0626564, 'lng': 241.7999823}}, 'CDF': {'Address': 'Campus Development & Facilities Bldg., 3434 Grand Ave.', 'Coordinates': {'lat': 34.02027959999999, 'lng': 241.7239611}}, 'CEM': {'Address': 'Ctr. for Electron Microscopy and Microanalysis, 814 W. Bloom Walk', 'Coordinates': {'lat': 34.0193248, 'lng': 241.7133413}}, 'CEN': {'Address': 'Centennial Apts., 2390 Portland St.', 'Coordinates': {'lat': 34.0313733, 'lng': 241.71782919999998}}, 'CFH': {'Address': 'Dean Bartlett Cromwell Field House, 3525 Watt Way', 'Coordinates': {'lat': 34.0225474, 'lng': 241.7115485}}, 'CHP': {'Address': 'Ctr. for Health Professions, 1540 Alcazar St.', 'Coordinates': {'lat': 37.3830519, 'lng': 354.0097743}}, 'CIC': {'Address': 'Jessica and Charles Cale and Ray Irani Residential College, 929 W. Jefferson Blvd.', 'Coordinates': {'lat': 34.0245517, 'lng': 241.7144791}}, 'CLH': {'Address': 'College House, 823 W. 34th St.', 'Coordinates': {'lat': 34.0231381, 'lng': 241.71546030000002}}, 'CNG': {'Address': 'Cardinal ’N Gold, 737 W. 30th St.', 'Coordinates': {'lat': 34.0263592, 'lng': 241.7193509}}, 'COH': {'Address': 'Cockins House, 2653 S. Hoover St.', 'Coordinates': {'lat': 34.0223519, 'lng': 241.714883}}, 'COL': {'Address': 'College Residence Hall, 615 W. McCarthy Way', 'Coordinates': {'lat': 34.0205064, 'lng': 241.7188013}}, 'CRC': {'Address': 'A.C. Allen Cowlings and Ilium Residential College, 3131 S. Hoover St.', 'Coordinates': {'lat': 34.026076, 'lng': 241.71533549999998}}, 'CRL': {'Address': 'Cancer Research Laboratory, 1303 Mission Rd.', 'Coordinates': {'lat': 39.084611, 'lng': 275.504885}}, 'CSA': {'Address': 'Clinical Sciences Annex, 2250 Alcazar St.', 'Coordinates': {'lat': 34.06245, 'lng': 241.79985299999998}}, 'CSC': {'Address': 'Clinical Science Ctr., 2250 Alcazar St.', 'Coordinates': {'lat': 34.06245, 'lng': 241.79985299999998}}, 'CST': {'Address': 'CAD Services Trailer, Berth 194 Yacht St., Wilmington', 'Coordinates': {'lat': 42.5481714, 'lng': 288.8275533}}, 'CTV': {'Address': 'Carson Television Ctr., 3450 Watt Way', 'Coordinates': {'lat': 34.0234263, 'lng': 241.7138423}}, 'CUB': {'Address': 'Credit Union Building, 3720 S. Flower St.', 'Coordinates': {'lat': 34.0172111, 'lng': 241.71894509999998}}, 'DCC': {'Address': 'Davidson Continuing Education Conference Ctr., 3415 S. Figueroa St.', 'Coordinates': {'lat': 34.0216556, 'lng': 241.7194305}}, 'DCF': {'Address': 'Delta Chi fraternity, 920 W. 28th St.', 'Coordinates': {'lat': 34.0282305, 'lng': 241.71701710000002}}, 'DEN': {'Address': 'Norris Dental Science Ctr., 925 W. 34th St.', 'Coordinates': {'lat': 34.0239888, 'lng': 241.7135585}}, 'DML': {'Address': 'Doheny Memorial Library, 3550 Trousdale Pkwy.', 'Coordinates': {'lat': 34.0201426, 'lng': 241.7162637}}, 'DMT': {'Address': 'Marks Tower, 612 W. Hellman Way', 'Coordinates': {'lat': 34.0197507, 'lng': 241.717852}}, 'DNI': {'Address': 'Dornsife Neuroscience Imaging Center, 3620-A McClintock Ave.', 'Coordinates': {'lat': 34.0214389, 'lng': 241.71097029999999}}, 'DOH': {'Address': 'Estelle Doheny Eye Foundation, 1355 San Pablo St.', 'Coordinates': {'lat': 34.060979, 'lng': 241.795911}}, 'DRB': {'Address': 'Denney Research Ctr., 1042 Downey Way', 'Coordinates': {'lat': 34.0214331, 'lng': 241.70885529999998}}, 'ECT': {'Address': 'Early Childhood Training Ctr., 741 W. 27th St.', 'Coordinates': {'lat': 34.0287447, 'lng': 241.7201504}}, 'EDM': {'Address': 'Edmondson Research Bldg., 1840 N. Soto St.', 'Coordinates': {'lat': 34.0651932, 'lng': 241.804085}}, 'EEB': {'Address': 'Hughes Aircraft Electrical Engineering Center, 3740 McClintock Ave.', 'Coordinates': {'lat': 34.0197157, 'lng': 241.7100013}}, 'EGG': {'Address': 'Egg Company II, 746 W. Adams Blvd.', 'Coordinates': {'lat': 34.0289258, 'lng': 241.7197568}}, 'EPS': {'Address': 'Figueroa Street Structure, 3533 S. Flower St.', 'Coordinates': {'lat': 34.0193678, 'lng': 241.71979670000002}}, 'ESH': {'Address': 'Engemann Student Health Center, 1031 W. 34th St.', 'Coordinates': {'lat': 34.0252529, 'lng': 241.7110734}}, 'EVK': {'Address': 'EVK Residence Hall, 635 W. McCarthy Way', 'Coordinates': {'lat': 34.0214704, 'lng': 241.7177886}}, 'FIG': {'Address': 'Figueroa Bldg., 3535 S. Figueroa St.', 'Coordinates': {'lat': 34.0198989, 'lng': 241.7183785}}, 'FLT': {'Address': 'Fluor Tower, 1027 W. 34th St.', 'Coordinates': {'lat': 34.0249027, 'lng': 241.71170890000002}}, 'FMH': {'Address': 'Forthmann House, 2801 South Hoover St.', 'Coordinates': {'lat': 34.0291659, 'lng': 241.715702}}, 'FML': {'Address': 'Feuchtwanger Memorial Library, 520 Paseo Miramar, Pacific Palisades', 'Coordinates': {'lat': 34.0462586, 'lng': 241.4440694}}, 'FMT': {'Address': 'Fairmont Apts., 2629 Portland St.', 'Coordinates': {'lat': 34.0301789, 'lng': 241.7170527}}, 'FPM': {'Address': 'Facilities Management & Planning Trailer, 3450 S. Vermont Ave.', 'Coordinates': {'lat': 34.02500819999999, 'lng': 241.7085294}}, 'FRE': {'Address': 'Freeman House, 1962 Glencoe Way', 'Coordinates': {'lat': 34.1056359, 'lng': 241.661418}}, 'FSA': {'Address': 'Founders Apts., 2610 Portland St.', 'Coordinates': {'lat': 34.030306, 'lng': 241.7179433}}, 'GAH': {'Address': 'Gamble House, 4 Westmoreland Pl., Pasadena', 'Coordinates': {'lat': 34.1516331, 'lng': 241.8390566}}, 'GAP': {'Address': 'Galen Athletic Pavilion, 3400 S. Figueroa St.', 'Coordinates': {'lat': 34.0209836, 'lng': 241.7201189}}, 'GEC': {'Address': 'Galen Event Center, 3400 S. Figueroa St.', 'Coordinates': {'lat': 34.0209836, 'lng': 241.7201189}}, 'GEH': {'Address': 'Max Kade Ctr. I, 2714 S. Hoover St.', 'Coordinates': {'lat': 34.0298232, 'lng': 241.7163219}}, 'GER': {'Address': 'Gerontology Ctr., Andrus, 3715 McClintock Ave.', 'Coordinates': {'lat': 34.0200617, 'lng': 241.7092749}}, 'GEX': {'Address': 'Max Kade Ctr. II, 2718 S. Hoover St.', 'Coordinates': {'lat': 34.0298232, 'lng': 241.7163219}}, 'GFS': {'Address': 'Grace Ford Salvatori Hall of Letters, Arts and Sciences, 3601 Watt Way', 'Coordinates': {'lat': 34.0213012, 'lng': 241.7119472}}, 'GLB': {'Address': 'General Labs Bldg., 1129 N. State St.', 'Coordinates': {'lat': 41.9027547, 'lng': 272.37173310000003}}, 'GNH': {'Address': 'County General Hospital, 1937 Hospital Pl.', 'Coordinates': {'lat': 35.6365424, 'lng': 271.1681476}}, 'HAP': {'Address': 'Helena Apts., 1220 W. 28th St.', 'Coordinates': {'lat': 34.0290501, 'lng': 241.7128507}}, 'HAR': {'Address': 'Mary Ormerod Harris Hall, Quinn Wing and Fisher Gallery, 823-29 Exposition Blvd.', 'Coordinates': {'lat': 38.5977207, 'lng': 238.55390979999999}}, 'HCB': {'Address': 'CMSC Hyperbaric Chamber Building, P.O. Box 398, Catalina Island, Avalon', 'Coordinates': {'lat': 33.3428193, 'lng': 241.6717721}}, 'HC1': {'Address': 'USC Health Care Consultation Ctr., 1510 San Pablo St.', 'Coordinates': {'lat': 34.0623826, 'lng': 241.7980061}}, 'HC2': {'Address': 'Health Care Consultation Ctr. 2, 1520 San Pablo St.', 'Coordinates': {'lat': 34.0623826, 'lng': 241.7980061}}, 'HC3': {'Address': 'Norris Health Care Consultation Ctr., 2204 Alcazar St.', 'Coordinates': {'lat': 37.3830519, 'lng': 354.0097743}}, 'HC4': {'Address': 'Estelle Doheny Eye Institute, 1537 Norfolk St.', 'Coordinates': {'lat': 34.060979, 'lng': 241.795911}}, 'HED': {'Address': 'HEDCO Petroleum & Chemical Engineering Bldg., 925 Bloom Walk', 'Coordinates': {'lat': 34.0199159, 'lng': 241.711616}}, 'HER': {'Address': 'Heritage Hall, 3501 Watt Way', 'Coordinates': {'lat': 34.0225539, 'lng': 241.713214}}, 'HHR': {'Address': 'Honors House, 2710 Severance St.', 'Coordinates': {'lat': 34.0288052, 'lng': 241.7186057}}, 'HIL': {'Address': 'Hillview Apts., 2605 Severance St.', 'Coordinates': {'lat': 34.0300085, 'lng': 241.7186618}}, 'HJF': {'Address': 'Howard Jones Football Practice Field, 3515 McClintock Ave.', 'Coordinates': {'lat': 34.0229945, 'lng': 241.7111241}}, 'HMR': {'Address': 'Hoffman Medical Research Ctr., 2011 Zonal Ave.', 'Coordinates': {'lat': 34.0599673, 'lng': 241.794755}}, 'HNB': {'Address': 'HEDCO Neurosciences Bldg., 3641 Watt Way', 'Coordinates': {'lat': 34.0223519, 'lng': 241.714883}}, 'HOH': {'Address': 'Hoffman Hall of Business Administration, 701 Exposition Blvd.', 'Coordinates': {'lat': 34.0187226, 'lng': 241.714766}}, 'HPB': {'Address': 'Huntington Pavilion Bldg., 625 Fair Oaks Ave., Suite 400, Pasadena', 'Coordinates': {'lat': 34.1348692, 'lng': 241.84930450000002}}, 'HRA': {'Address': 'Health Research Assn., 1640 Marengo St.', 'Coordinates': {'lat': 34.0593258, 'lng': 241.78629560000002}}, 'HRH': {'Address': 'Harris Residence Hall, 634 W. 34th St.', 'Coordinates': {'lat': 34.0215368, 'lng': 241.7182457}}, 'HSA': {'Address': 'Health Sciences Alhambra, 1000 S. Fremont Ave., Alhambra, CA', 'Coordinates': {'lat': 34.0819066, 'lng': 241.850014}}, 'HSH': {'Address': 'Hazel and Stanley Hall Financial Services Bldg., 851 Downey Way', 'Coordinates': {'lat': 34.02031729999999, 'lng': 241.7128689}}, 'HSP': {'Address': 'Health Sciences Campus Parking Structure, 1334 Biggy St.', 'Coordinates': {'lat': 34.0608547, 'lng': 241.79357720000002}}, 'HSR': {'Address': 'Hoover Street Residence, 2827 Hoover St.', 'Coordinates': {'lat': 34.0288945, 'lng': 241.71583270000002}}, 'HUC': {'Address': 'Hebrew Union College, 3077 University Ave.', 'Coordinates': {'lat': 34.025737, 'lng': 241.7169131}}, 'IFT': {'Address': 'Ickovics Family Trust Bldg. Fine Arts, 3001 S. Flower St.', 'Coordinates': {'lat': 34.0243266, 'lng': 241.7227566}}, 'IRC': {'Address': 'Internationally Themed Residential College, 3771 McClintock Ave.', 'Coordinates': {'lat': 34.0193746, 'lng': 241.7090321}}, 'IRD': {'Address': 'Interns & Residents Dormitory, 2020 Zonal Ave.', 'Coordinates': {'lat': 34.0590574, 'lng': 241.79343369999998}}, 'ISE': {'Address': 'Information Science Institute East, 3811 N. Fairfax Dr., Arlington, VA', 'Coordinates': {'lat': 38.8827394, 'lng': 282.8940033}}, 'ISI': {'Address': 'Information Science Institute, 4676 Admiralty Way, Marina del Rey', 'Coordinates': {'lat': 33.9800707, 'lng': 241.5599615}}, 'IYH': {'Address': 'USC Jimmy Iovine and Andre Young Hall, 3738 Watt Way', 'Coordinates': {'lat': 34.0223519, 'lng': 241.714883}}, 'JEF': {'Address': 'Jefferson Bldg., 950 W. Jefferson Blvd.', 'Coordinates': {'lat': 34.0248257, 'lng': 241.7129997}}, 'JEP': {'Address': 'Joint Educational Project House, 801 W. 34th St.', 'Coordinates': {'lat': 34.0228532, 'lng': 241.7160204}}, 'JFF': {'Address': 'Jill and Frank Fertitta Hall, 610 Childs Way', 'Coordinates': {'lat': 34.0186039, 'lng': 241.7175115}}, 'JKP': {'Address': 'Jane Popovich and J. Kristoffer Popovich Hall, 611 Exposition Blvd.', 'Coordinates': {'lat': 34.0187469, 'lng': 241.7170418}}, 'JMC': {'Address': 'John McKay Ctr., 940 W. 35th St.', 'Coordinates': {'lat': 34.0230555, 'lng': 241.71235919999998}}, 'JWS': {'Address': 'John Williams Scoring Stage, 3450 Watt Way', 'Coordinates': {'lat': 34.0228166, 'lng': 241.7144004}}, 'KAM': {'Address': 'Keith Administration & Medical Forum Bldg.; Mayer Medical Teaching Ctr., 1975 Zonal Ave.', 'Coordinates': {'lat': 34.0610191, 'lng': 241.79449599999998}}, 'KAP': {'Address': 'Kaprielian Hall, 3620 S. Vermont Ave.', 'Coordinates': {'lat': 34.02240889999999, 'lng': 241.70898640000001}}, 'KCH': {'Address': 'Kerckhoff Carriage House, 734 W. Adams Blvd.', 'Coordinates': {'lat': 34.0290711, 'lng': 241.7203783}}, 'KDC': {'Address': 'Glorya Kaufman International Dance Center, 849 W. 34th St.', 'Coordinates': {'lat': 34.023499, 'lng': 241.714699}}, 'KER': {'Address': 'Kerckhoff Hall, 734 W. Adams Blvd.', 'Coordinates': {'lat': 34.0290711, 'lng': 241.7203783}}, 'KSH': {'Address': 'University Club at King Stoops Hall, 705 W. 34th St.', 'Coordinates': {'lat': 34.0226087, 'lng': 241.7168683}}, 'LAB': {'Address': 'La Sorbonne Apts., 1170 W. 31st St.', 'Coordinates': {'lat': 34.0261853, 'lng': 241.7114729}}, 'LAC': {'Address': 'Los Angeles County + USC Hospital, 1200 N. State St.', 'Coordinates': {'lat': 34.0602202, 'lng': 241.7896959}}, 'LAW': {'Address': 'Elvon and Mabel Musick Law Bldg., 699 Exposition Blvd.', 'Coordinates': {'lat': 34.0186748, 'lng': 241.7156631}}, 'LCA': {'Address': 'Lambda Chi Alpha fraternity, 720 W. 28th St.', 'Coordinates': {'lat': 40.753097, 'lng': 285.9927203}}, 'LHI': {'Address': 'Donald P. and Katherine B. Loker Hydrocarbon Inst., 837 Bloom Walk', 'Coordinates': {'lat': 34.0197187, 'lng': 241.7121403}}, 'LJS': {'Address': 'Laird J. Stabler Memorial Hall, 840 Downey Way', 'Coordinates': {'lat': 34.0200628, 'lng': 241.7127665}}, 'LRA': {'Address': 'Livingston Research Annex, 1321 N. Mission Rd', 'Coordinates': {'lat': 34.0634599, 'lng': 241.7892507}}, 'LRB': {'Address': 'Livingston Research Building, 1321 N. Mission Rd', 'Coordinates': {'lat': 34.0637094, 'lng': 241.7894725}}, 'LRC': {'Address': 'General William Lyon University Ctr., 1026 W. 34th St.', 'Coordinates': {'lat': 40.7527048, 'lng': 286.0053297}}, 'LTS': {'Address': 'Katherine B. Loker Track Stadium, 3550 McClintock Ave.', 'Coordinates': {'lat': 34.0226439, 'lng': 241.7116404}}, 'LVL': {'Address': 'Thomas and Dorothy Leavey Library, 650 W. McCarthy Way', 'Coordinates': {'lat': 34.0218002, 'lng': 241.7171704}}, 'MAB': {'Address': 'Manor Apts., 2636 Portland St.', 'Coordinates': {'lat': 34.0297123, 'lng': 241.7176694}}, 'MBC': {'Address': 'MacDonald Becket Center, 850 W. Bloom Walk', 'Coordinates': {'lat': 42.2991669, 'lng': 286.7591983}}, 'MCA': {'Address': 'McKibben Addition, 1333 San Pablo St.', 'Coordinates': {'lat': 34.0604564, 'lng': 241.795492}}, 'MCB': {'Address': 'Michelson Center For Convergent Bioscience, 1002 Childs Way', 'Coordinates': {'lat': 34.0217334, 'lng': 241.7104736}}, 'MCC': {'Address': 'McClintock Building, 1010 W. Jefferson Blvd.', 'Coordinates': {'lat': 34.0249043, 'lng': 241.7126432}}, 'MCH': {'Address': 'McKibben Hall, 1333 San Pablo St.', 'Coordinates': {'lat': 34.0602502, 'lng': 241.7954039}}, 'MCT': {'Address': 'McCulloch Townhomes, 953 W. 30th St.', 'Coordinates': {'lat': 34.0278792, 'lng': 241.7163615}}, 'MHC': {'Address': 'The Kathleen L. McCarthy Honors College, 3096 S. McClintock Ave.', 'Coordinates': {'lat': 34.0257584, 'lng': 241.714395}}, 'MHP': {'Address': 'Mudd Memorial Hall of Philosophy, 3709 Trousdale Pkwy.', 'Coordinates': {'lat': 34.0187985, 'lng': 241.7133795}}, 'MMR': {'Address': 'Mudd Memorial Research Bldg., 1333 San Pablo St.', 'Coordinates': {'lat': 34.0604564, 'lng': 241.795492}}, 'MOL': {'Address': 'Medical Oncology Lab, 1237 N. Mission Rd.', 'Coordinates': {'lat': 34.0624252, 'lng': 241.78787979999998}}, 'MRC': {'Address': 'Priam Residential College, 835 W. Jefferson Blvd.', 'Coordinates': {'lat': 34.0245517, 'lng': 241.7144791}}, 'MRF': {'Address': 'Montgomery Ross Fisher Bldg., 669 W. 34th St.', 'Coordinates': {'lat': 34.022333, 'lng': 241.7174566}}, 'MRI': {'Address': 'Magnetic Resonance Imaging Building, 2025 Zonal Ave.', 'Coordinates': {'lat': 34.0597844, 'lng': 241.795208}}, 'MSC': {'Address': 'Marine Sciences Ctr. Lab, Catalina Island', 'Coordinates': {'lat': 33.4447276, 'lng': 241.51652810000002}}, 'MSR': {'Address': 'Marine Sciences Center Residence Hall, Catalina Island', 'Coordinates': {'lat': 33.4447276, 'lng': 241.51652810000002}}, 'MTX': {'Address': 'Marks Tennis Stadium, 1075 Childs Way', 'Coordinates': {'lat': 34.0231369, 'lng': 241.7088134}}, 'MUS': {'Address': 'Albert S. Raubenheimer Music Faculty Memorial Bldg., 840 W. 34th St.', 'Coordinates': {'lat': 34.0229518, 'lng': 241.714905}}, 'NBC': {'Address': 'Shelly and Ofer Nemirovsky and David C. Bohnett Residential College, 3201 S. Hoover St.', 'Coordinates': {'lat': 34.0248873, 'lng': 241.71581020000002}}, 'NCT': {'Address': 'Norris Cinema Theatre, 3507 Trousdale Pkwy.', 'Coordinates': {'lat': 34.0221707, 'lng': 241.7145893}}, 'NLC': {'Address': 'Norman Lear Center, 8383 Wilshire Blvd. #652', 'Coordinates': {'lat': 34.0650997, 'lng': 241.6266379}}, 'NML': {'Address': 'Norris Medical Library, 2003 Zonal Ave.', 'Coordinates': {'lat': 34.0609243, 'lng': 241.7951108}}, 'NOR': {'Address': 'Kenneth Norris Jr. Cancer Hospital Research Institute, 1441 Eastlake Ave.', 'Coordinates': {'lat': 34.0612971, 'lng': 241.79533859999998}}, 'NRT': {'Address': 'Harlyne J. Norris Research Tower, 1450 Biggy St.', 'Coordinates': {'lat': 34.0621274, 'lng': 241.7949586}}, 'NTT': {'Address': 'Dr. Norman Topping Tower, 1441 Eastlake Ave.', 'Coordinates': {'lat': 34.0615314, 'lng': 241.794576}}, 'OCC': {'Address': 'Orange County Campus, 2300 Michelson Dr., Irvine, 92714', 'Coordinates': {'lat': 33.676106, 'lng': 242.146183}}, 'OCW': {'Address': 'Harold E. and Lillian M. Moulton Organic Chemistry Wing, 810 Downey Way', 'Coordinates': {'lat': 34.019488, 'lng': 241.7137196}}, 'OHE': {'Address': 'Olin Hall of Engineering, 3650 McClintock Ave.', 'Coordinates': {'lat': 34.0206254, 'lng': 241.7103063}}, 'ONE': {'Address': 'One Institute, 909 W. Adams Blvd.', 'Coordinates': {'lat': 34.0310318, 'lng': 241.7184344}}, 'OSP': {'Address': 'Our Savior Parish and USC Caruso Catholic Center, 844 W. 32nd St.', 'Coordinates': {'lat': 34.0251959, 'lng': 241.7165779}}, 'PAM': {'Address': 'Pacific Asia Museum, 46 N. Los Robles Ave., Pasadena', 'Coordinates': {'lat': 34.146669, 'lng': 241.8589741}}, 'PCA': {'Address': 'Pacific Apts., 2637 Severance St.', 'Coordinates': {'lat': 34.029471, 'lng': 241.7181706}}, 'PCE': {'Address': 'Petroleum and Chemical Engineering Bldg., 923 Bloom Walk', 'Coordinates': {'lat': 34.0200908, 'lng': 241.7107497}}, 'PED': {'Address': 'Physical Education Bldg., 3560 Watt Way', 'Coordinates': {'lat': 34.0212774, 'lng': 241.71368130000002}}, 'PHE': {'Address': 'Powell Hall, 3737 Watt Way', 'Coordinates': {'lat': 34.0191533, 'lng': 241.7112286}}, 'PHH': {'Address': 'Phinney Hall, 1721 Griffin Ave.', 'Coordinates': {'lat': 34.0634163, 'lng': 241.78821440000002}}, 'PKS': {'Address': 'Parkside Apts., 3730 S. McClintock Ave.', 'Coordinates': {'lat': 34.0192298, 'lng': 241.7098517}}, 'PSA': {'Address': 'Downey Way Structure, Lot 33, 3667 McClintock Ave.', 'Coordinates': {'lat': 34.0210461, 'lng': 241.7096613}}, 'PSB': {'Address': 'Jefferson Boulevard Structure, Lot 1, 1150 Jefferson Blvd.', 'Coordinates': {'lat': 34.0248092, 'lng': 241.710579}}, 'PSC': {'Address': 'John Stauffer Pharmaceutical Sciences Ctr., 1985 Zonal Ave.', 'Coordinates': {'lat': 34.0602352, 'lng': 241.79399089999998}}, 'PSD': {'Address': 'Royal Street Structure, 649 W. 34th St.', 'Coordinates': {'lat': 40.756234, 'lng': 285.9971429}}, 'PSO': {'Address': 'Flower Street Structure, 3701 S. Flower St.', 'Coordinates': {'lat': 34.01769, 'lng': 241.718088}}, 'PSX': {'Address': 'McCarthy Way Structure, 620 W. McCarthy Way', 'Coordinates': {'lat': 34.0205212, 'lng': 241.7175379}}, 'PTD': {'Address': 'Pardee Tower, 614 Hellman Way', 'Coordinates': {'lat': 34.019997, 'lng': 241.71739530000002}}, 'PVB': {'Address': 'Playa Vista Bldg., 12015 E. Waterfront Dr., Playa Vista', 'Coordinates': {'lat': 33.9831056, 'lng': 241.5958731}}, 'PVT': {'Address': 'Playa Vista Bldg. Two, 12025 Waterfront Dr., Playa Vista', 'Coordinates': {'lat': 33.9828391, 'lng': 241.5952939}}, 'RAN': {'Address': 'Hoffman Contracts Research Bldg., 3716 S. Hope St.', 'Coordinates': {'lat': 34.0162121, 'lng': 241.7208928}}, 'RGA': {'Address': 'Regent Apts., 1138 W. 29th St.', 'Coordinates': {'lat': 34.0281212, 'lng': 241.7149465}}, 'RGL': {'Address': 'Ralph and Goldy Lewis Hall, 650 Childs Way', 'Coordinates': {'lat': 34.0191777, 'lng': 241.7164194}}, 'RHM': {'Address': 'Ramo Hall of Music, 830 W. 34th St.', 'Coordinates': {'lat': 34.0227642, 'lng': 241.7153188}}, 'RHR': {'Address': 'Radisson Hotel Restaurant, 3520 S. Figueroa St.', 'Coordinates': {'lat': 34.0199035, 'lng': 241.71881430000002}}, 'RMH': {'Address': 'Radisson Midcity Hotel, 3540 S. Figueroa St.', 'Coordinates': {'lat': 34.0194059, 'lng': 241.718974}}, 'RRB': {'Address': 'Rapp Engineering Research Bldg., 854 Downey Way', 'Coordinates': {'lat': 34.0200437, 'lng': 241.71251189999998}}, 'RRI': {'Address': 'Ray R. Irani Hall, 1050 Childs Way', 'Coordinates': {'lat': 34.0222018, 'lng': 241.7098525}}, 'RTA': {'Address': 'Regal Trojan Apts., 870 W. Adams Blvd.', 'Coordinates': {'lat': 34.03017270000001, 'lng': 241.7182144}}, 'RTH': {'Address': 'Ronald Tutor Hall of Engineering, 3710 McClintock Ave.', 'Coordinates': {'lat': 34.0201366, 'lng': 241.7100743}}, 'RZC': {'Address': 'Robert Zemeckis Ctr. for Digital Arts, 3131 S. Figueroa St.', 'Coordinates': {'lat': 34.0241643, 'lng': 241.7207045}}, 'SAC': {'Address': 'Sacramento Center, 1800 I Street, Sacramento, CA', 'Coordinates': {'lat': 38.5781247, 'lng': 238.5180339}}, 'SAI': {'Address': 'Stardust Apts., 634 W. 27th St.', 'Coordinates': {'lat': 34.0273926, 'lng': 241.72200850000002}}, 'SAL': {'Address': 'Salvatori Computer Science Ctr., 941 Bloom Walk', 'Coordinates': {'lat': 34.019476, 'lng': 241.71052609999998}}, 'SCA': {'Address': 'School of Cinematic Arts, 900 W. 34th St.', 'Coordinates': {'lat': 34.0234463, 'lng': 241.7128507}}, 'SCB': {'Address': 'School of Cinematic Arts Bldg. B, 930 W. 34th St.', 'Coordinates': {'lat': 34.0234463, 'lng': 241.7128507}}, 'SCC': {'Address': 'School of Cinematic Arts Bldg. C, 935 W. 35th St.', 'Coordinates': {'lat': 34.0234463, 'lng': 241.7128507}}, 'SCD': {'Address': 'Scene Dock (Theatre), 1030 W. 37th St.', 'Coordinates': {'lat': 34.0207382, 'lng': 241.7088224}}, 'SCE': {'Address': 'School of Cinematic Arts Bldg. E, 905 W. 35th St.', 'Coordinates': {'lat': 34.0234463, 'lng': 241.7128507}}, 'SCI': {'Address': 'School of Cinematic Arts Bldg. I, 3470 McClintock Ave.', 'Coordinates': {'lat': 34.0240226, 'lng': 241.7124954}}, 'SCO': {'Address': 'USC Building One, 1149 S. Hill St.', 'Coordinates': {'lat': 34.0391822, 'lng': 241.73870390000002}}, 'SCP': {'Address': 'South Coast Plaza Bldg., 3333 Bristol St., Costa Mesa', 'Coordinates': {'lat': 33.69100299999999, 'lng': 242.1110233}}, 'SCS': {'Address': 'School of Cinematic Arts Station, 639 W. 32nd St.', 'Coordinates': {'lat': 34.0242004, 'lng': 241.7197276}}, 'SCT': {'Address': 'USC Building Two, 1150 S. Olive St.', 'Coordinates': {'lat': 34.0395199, 'lng': 241.7382928}}, 'SCX': {'Address': 'School of Cinematic Arts Bldg. D, 915 W. 35th St.', 'Coordinates': {'lat': 34.0234463, 'lng': 241.7128507}}, 'SGA': {'Address': 'Seven Gables Apts., 620-626 W. 30th St.', 'Coordinates': {'lat': 34.0251805, 'lng': 241.72100360000002}}, 'SGM': {'Address': 'Seeley G. Mudd Bldg., 3620 McClintock Ave.', 'Coordinates': {'lat': 34.0214389, 'lng': 241.71097029999999}}, 'SHN': {'Address': 'Stevens Hall Neuroimaging, 2025 Zonal Ave.', 'Coordinates': {'lat': 34.0597165, 'lng': 241.7953489}}, 'SHP': {'Address': 'Shrine Parking Structure, 645 W. Jefferson Blvd.', 'Coordinates': {'lat': 34.0234928, 'lng': 241.7186514}}, 'SHS': {'Address': 'Stauffer Hall of Science, 835 Bloom Walk', 'Coordinates': {'lat': 34.0195725, 'lng': 241.7124185}}, 'SIE': {'Address': 'Sierra Apts., 2638 Portland St.', 'Coordinates': {'lat': 34.0296491, 'lng': 241.71723079999998}}, 'SKS': {'Address': 'Steven and Kathryn Sample Hall, 3607 Trousdale Pkwy.', 'Coordinates': {'lat': 34.0199943, 'lng': 241.7142026}}, 'SLH': {'Address': 'Stauffer Science Lecture Hall, 831 Bloom Walk', 'Coordinates': {'lat': 34.0195725, 'lng': 241.7124185}}, 'SMF': {'Address': 'Soni-McAlister Field, 3000 S. Hoover St.', 'Coordinates': {'lat': 34.02657730000001, 'lng': 241.7172435}}, 'SNA': {'Address': 'Senator Apts., 1101-1109 W. 28th St.', 'Coordinates': {'lat': 34.02950240000001, 'lng': 241.7157053}}, 'SOS': {'Address': 'Social Science Bldg., 3502 Trousdale Pkwy.', 'Coordinates': {'lat': 34.0215692, 'lng': 241.716248}}, 'SPA': {'Address': 'Shrine Place Apartments, 3015-3017 Shrine Pl.', 'Coordinates': {'lat': 34.0249725, 'lng': 241.72025}}, 'SPS': {'Address': 'San Pablo Parking Structure, 1630 N. San Pablo St.', 'Coordinates': {'lat': 37.8083304, 'lng': 237.7279358}}, 'SRH': {'Address': 'Seaver Residence Hall, 1969 Zonal Ave.', 'Coordinates': {'lat': 34.0606447, 'lng': 241.79426619999998}}, 'SSA': {'Address': 'Severance St. Apts., 2630 Severance St.', 'Coordinates': {'lat': 34.0293796, 'lng': 241.7189026}}, 'SSB': {'Address': 'Soto Street Building, 2001 N. Soto St.', 'Coordinates': {'lat': 34.0666261, 'lng': 241.8025301}}, 'SSC': {'Address': 'Seaver Science Ctr., 920 Bloom Walk', 'Coordinates': {'lat': 34.0198968, 'lng': 241.7105967}}, 'SSH': {'Address': 'Severance St. House, 2716 Severance St.', 'Coordinates': {'lat': 34.0286939, 'lng': 241.7184001}}, 'SSL': {'Address': 'Seaver Science Library, 910 Bloom Walk', 'Coordinates': {'lat': 34.0196113, 'lng': 241.71120059999998}}, 'SST': {'Address': 'Soto Street Building Two, 2011 Soto St.', 'Coordinates': {'lat': 34.0668887, 'lng': 241.8032061}}, 'STO': {'Address': 'Stonier Hall, 837 Downey Way', 'Coordinates': {'lat': 34.0223519, 'lng': 241.714883}}, 'STU': {'Address': 'Gwynn Wilson Student Union, 3601 Trousdale Pkwy.', 'Coordinates': {'lat': 34.0202225, 'lng': 241.7143572}}, 'SUN': {'Address': 'Sunset Apts., 1144 W. 29th St.', 'Coordinates': {'lat': 34.028125, 'lng': 241.7148375}}, 'SWC': {'Address': 'Social Work Ctr., 655 W. 34th St.', 'Coordinates': {'lat': 34.022333, 'lng': 241.717516}}, 'TAP': {'Address': 'Troyland Apts., 955-959 W. Adams Blvd.', 'Coordinates': {'lat': 34.0315566, 'lng': 241.716831}}, 'TCC': {'Address': 'Ronald Tutor Campus Center, 3607 Trousdale Pkwy.', 'Coordinates': {'lat': 34.02017379999999, 'lng': 241.7135139}}, 'TGF': {'Address': 'Town and Gown, 665 Exposition Blvd.', 'Coordinates': {'lat': 34.0191157, 'lng': 241.7157516}}, 'THH': {'Address': 'Taper Hall of Humanities, 3501 Trousdale Pkwy.', 'Coordinates': {'lat': 34.0222316, 'lng': 241.7154381}}, 'THR': {'Address': 'Two Harbors Residences, Two Harbors, Catalina Island', 'Coordinates': {'lat': 33.44, 'lng': 241.50167}}, 'TMC': {'Address': 'The Music Complex, 3450 Watt Way', 'Coordinates': {'lat': 34.0211693, 'lng': 241.7127928}}, 'TOW': {'Address': 'Tower Hall, 1711 N. Griffin Ave.', 'Coordinates': {'lat': 34.0737037, 'lng': 241.7882618}}, 'TPA': {'Address': 'Twin Palms Apts., 2635 Portland St.', 'Coordinates': {'lat': 34.0299387, 'lng': 241.7170527}}, 'TRF': {'Address': 'Temporary Research Facility, 3430 S. Vermont Ave.', 'Coordinates': {'lat': 34.0251379, 'lng': 241.7094443}}, 'TRH': {'Address': 'Troy Hall, 3025 Royal St.', 'Coordinates': {'lat': 34.0253425, 'lng': 241.7177193}}, 'TRO': {'Address': 'Trojan Residence Hall, 615 Childs Way', 'Coordinates': {'lat': 34.01967, 'lng': 241.7175508}}, 'TSA': {'Address': 'Terrace Apts., 2822-2830 Ellendale Pl.', 'Coordinates': {'lat': 34.031695, 'lng': 241.7114934}}, 'TTL': {'Address': 'Technical Theatre Lab., 1020 Bloom Walk', 'Coordinates': {'lat': 38.8992276, 'lng': 282.9773748}}, 'TUS': {'Address': 'Tuscany, 3760 S. Figueroa St.', 'Coordinates': {'lat': 34.0168719, 'lng': 241.7174923}}, 'TYL': {'Address': 'Tyler Bldg., 3601 S. Flower St.', 'Coordinates': {'lat': 34.0181579, 'lng': 241.7187509}}, 'UAC': {'Address': 'Uytengsu Aquatics Center, 3441 McClintock Ave.', 'Coordinates': {'lat': 34.0239277, 'lng': 241.7115719}}, 'UGB': {'Address': 'University Gardens Building, 3500 S. Figueroa St.', 'Coordinates': {'lat': 34.019998, 'lng': 241.7192731}}, 'UGW': {'Address': 'University Gateway, 3335 S. Figueroa St.', 'Coordinates': {'lat': 34.02318959999999, 'lng': 241.7198846}}, 'UHP': {'Address': 'University Hosp. Parking, 1538 San Pablo St.', 'Coordinates': {'lat': 34.0627749, 'lng': 241.7970161}}, 'UNH': {'Address': 'USC University Hosp., 1500 San Pablo St.', 'Coordinates': {'lat': 34.06199429999999, 'lng': 241.7987393}}, 'UPX': {'Address': 'Grand Avenue Structure, 3401 S. Grand Ave.', 'Coordinates': {'lat': 34.0200103, 'lng': 241.7227844}}, 'URA': {'Address': 'University Regent Apts., 1219 W. 27th St.', 'Coordinates': {'lat': 34.0305939, 'lng': 241.71294210000002}}, 'URC': {'Address': 'University Religious Ctr., 835 W. 34th St.', 'Coordinates': {'lat': 34.0233542, 'lng': 241.7152362}}, 'URH': {'Address': 'University Residence Hall, 616 W. 34th St.', 'Coordinates': {'lat': 40.7558881, 'lng': 285.9976724}}, 'UUC': {'Address': 'United University Church, 817 W. 34th St.', 'Coordinates': {'lat': 34.0352933, 'lng': 241.7172583}}, 'UVO': {'Address': 'University Village One, 3015 S. Hoover St.', 'Coordinates': {'lat': 34.0254248, 'lng': 241.7148135}}, 'VBB': {'Address': 'Valley Blvd. Bldg., 4351 E. Valley Blvd.', 'Coordinates': {'lat': 34.0804554, 'lng': 241.9164075}}, 'VHE': {'Address': 'Vivian Hall of Engineering, 3651 Watt Way', 'Coordinates': {'lat': 34.0201249, 'lng': 241.71177849999998}}, 'VHH': {'Address': 'Verdugo Hills Hospital, 1812 Verdugo Blvd.', 'Coordinates': {'lat': 34.2048286, 'lng': 241.7842067}}, 'VIS': {'Address': 'Vista Apts., 2701 Severance St.', 'Coordinates': {'lat': 34.0292288, 'lng': 241.7180576}}, 'VKC': {'Address': 'Von KleinSmid Ctr. for International and Public Affairs, 3518 Trousdale Pkwy.', 'Coordinates': {'lat': 34.0212585, 'lng': 241.71599750000001}}, 'VPD': {'Address': 'Verna & Peter Dauterive Hall, 635 Downey Way', 'Coordinates': {'lat': 34.0189457, 'lng': 241.7161415}}, 'VWB': {'Address': 'Valley Warehouse Bldg., 4121 Valley Blvd.', 'Coordinates': {'lat': 34.064767, 'lng': 241.80457130000002}}, 'WAH': {'Address': 'Watt Hall of Architecture and Fine Arts, 850 Bloom Walk', 'Coordinates': {'lat': 34.0192264, 'lng': 241.71275889999998}}, 'WIN': {'Address': 'Windsor Apts., 1149 W. 28th St.', 'Coordinates': {'lat': 34.0296662, 'lng': 241.71467769999998}}, 'WPH': {'Address': 'Waite Phillips Hall, 3470 Trousdale Pkwy.', 'Coordinates': {'lat': 34.0219551, 'lng': 241.71622259999998}}, 'WTO': {'Address': 'Webb Tower, 1015 W. 34th St.', 'Coordinates': {'lat': 34.0246117, 'lng': 241.7123645}}, 'ZHS': {'Address': 'Zumberge Hall of Science, 3651 Trousdale Pkwy.', 'Coordinates': {'lat': 34.0191041, 'lng': 241.7135354}}, 'ZNI': {'Address': 'Zilkha Neurogenetic Institute, 1501 San Pablo St.', 'Coordinates': {'lat': 34.062439, 'lng': 241.7963406}}}
            # Splits building name from room number and sets loc to building name
            loc = loc.split(" ")[0]
            build = ""
            for char in loc:
                if char.isalpha():
                    build += char
                if build in directory.keys():
                    return {'Abbreviation': build, 'Coordinates': [directory[build]['Coordinates']['lat'], directory[build]['Coordinates']['lng']]}
        except AttributeError:
            print('Section', self.id, 'has no location, cannot complete combinations')
            sys.exit(1)

    def section_type(self):
        if self.type == "Lec":
            return "Lecture"
        elif self.type == "Dis":
            return "Discussion"
        elif self.type == "Lab":
            return "Lab"
        elif self.type == "Qz":
            return "Quiz"

    def days_of_week(self):
        if self.days == "M":
            return "Monday"
        elif self.days == "MW":
            return "Monday/Wednesday"
        elif self.days == "MWF":
            return "Monday/Wednesday/Friday"
        elif self.days == "T":
            return "Tuesday"
        elif self.days == "TH":
            return "Tuesday/Thursday"
        elif self.days == "W":
            return "Wednesday"
        elif self.days == "H":
            return "Thursday"
        elif self.days == "F":
            return "Friday"

In [0]:
#@title
class Course:
    def __init__(self, course_code):
        self.code = course_code
        self.sections = []
        self.segments = []
        self.professors = []
        self.combinations = []

    def add_section(self, section):
        self.sections.append(section)

    def dev_segment(self, segment, section, sec_type):
        sec_types = {'Dis': {'category' : 'Discs', 'type': 'Disc'}, 'Lab': {'category' : 'Labs', 'type': 'Lab'}, 'Qz': {'category': 'Quizzes', 'type': 'Quiz'}}
        if sec_types[sec_type]['category'] in segment.keys():
            segment[sec_types[sec_type]['category']].append({"id": section.id, "start": section.start, "end": section.end, "days": section.days, "type": sec_types[sec_type]['type'], "location": section.location, "code": self.code})
        else:
            segment[sec_types[sec_type]['category']] = [{"id": section.id, "start": section.start, "end": section.end, "days": section.days, "type": sec_types[sec_type]['type'], "location": section.location, "code": self.code}]
        return segment

    def organize(self, exclude_profs):
      # If there is only one section
        if len(self.sections) == 1:
            segment = {"Lecture": {"id": self.sections[0].id, "start": self.sections[0].start, "end": self.sections[0].end, "days": self.sections[0].days, "type": self.sections[0].type, "location": self.sections[0].location, "code": self.code,"instructor": self.sections[0].professor}}
            self.segments.append(segment)
        else:
            for section in self.sections:
                if section.type == "Lec":
                    segment = {"Lecture": {"id": section.id, "start": section.start, "end": section.end, "days": section.days, "type": section.type, "location": section.location, "code": self.code, "instructor": section.professor}}
                    lec_loc = self.sections.index(section)
                    # If the lecture section isn't the last element
                    if lec_loc < (len(self.sections) - 1):
                        temp_section = lec_loc + 1
                        temp_type = self.sections[temp_section].type
                        if temp_type == "Lec":
                            # Supp_section means supplementary section, meaning in addition to Lecture
                            for supp_section in self.sections:
                                if supp_section.type == "Dis":
                                    segment = self.dev_segment(segment, supp_section, "Dis")
                                elif supp_section.type == "Lab":
                                    segment = self.dev_segment(segment, supp_section, "Lab")
                                elif supp_section.type == "Qz":
                                    segment = self.dev_segment(segment, supp_section, "Qz")

                            self.segments.append(segment)

                        while (temp_section < len(self.sections)) and (temp_type != "Lec"):
                            if temp_type == "Dis":
                                segment = self.dev_segment(segment, self.sections[temp_section], "Dis")
                            elif temp_type == "Lab":
                                segment = self.dev_segment(segment, self.sections[temp_section], "Lab")
                            elif temp_type == "Qz":
                                segment = self.dev_segment(segment, self.sections[temp_section], "Qz")

                            temp_section += 1
                            if temp_section < len(self.sections):
                                temp_type = self.sections[temp_section].type

                        if (segment not in self.segments) and (segment["Lecture"]["instructor"] not in exclude_profs):
                            self.segments.append(segment)

                if not self.segments:
                    return "NPC"

    def add_combo(self, combo):
        self.combinations.append(combo)

    def __str__(self):
        desc = "\n" + self.code
        for section in self.sections:
            desc += "\n\t" + section.section_type() + " - " + section.id
            desc += "\n\t\tTimings: " + section.days_of_week() + " " + section.start.strftime('%H:%M') + "-" + section.end.strftime('%H:%M')
            desc += "\n\t\tProfessor: " + section.professor
        return desc

In [0]:
#@title
class Schedule:
    def __init__(self):
        self.combinations = []
        self.monday = []
        self.tuesday = []
        self.wednesday = []
        self.thursday = []
        self.friday = []
        self.codes = []

    @staticmethod
    def approved_times(pref_times, combo):
        if pref_times == {}:
            return True

        for key in combo:
            if key == "Quizzes":
                return True
            
            days = set([day for day in combo[key]['days']])

            if 'M' in days:
                if (combo[key]["start"] < pref_times["monday"]["start"]) or (combo[key]["end"] > pref_times["monday"]["end"]):
                    return False
            if 'T' in days:
                if (combo[key]["start"] < pref_times["tuesday"]["start"]) or (combo[key]["end"] > pref_times["tuesday"]["end"]):
                    return False
            if 'W' in days:
                if (combo[key]["start"] < pref_times["wednesday"]["start"]) or (combo[key]["end"] > pref_times["wednesday"]["end"]):
                    return False
            if 'H' in days:
                if (combo[key]["start"] < pref_times["thursday"]["start"]) or (combo[key]["end"] > pref_times["thursday"]["end"]):
                    return False
            if 'F' in days:
                if (combo[key]["start"] < pref_times["friday"]["start"]) or (combo[key]["end"] > pref_times["friday"]["end"]):
                    return False

        return True

    def add_combo(self, pref_times, combo):
        if self.combo_integrity(combo) and self.compat_combo(combo) and self.approved_times(pref_times, combo):
            self.combinations.append(combo)
            for key in combo:
                section = combo[key]
                if section["type"] == "Lec":
                    self.codes.append(section["code"])
                if section["days"] == "M":
                    self.monday.append(section)
                elif section["days"] == "MW":
                    self.monday.append(section)
                    self.wednesday.append(section)
                elif section["days"] == "MWF":
                    self.monday.append(section)
                    self.wednesday.append(section)
                    self.friday.append(section)
                elif section["days"] == "T":
                    self.tuesday.append(section)
                elif section["days"] == "TH":
                    self.tuesday.append(section)
                    self.thursday.append(section)
                elif section["days"] == "W":
                    self.wednesday.append(section)
                elif section["days"] == "H":
                    self.thursday.append(section)
                elif section["days"] == "F":
                    self.friday.append(section)

    @staticmethod
    def combo_integrity(combo):
        lec_days = combo['Lecture']['days']
        integrity = lambda section1, section2 : (section1["end"] < section2["start"]) or (section2["end"] < section1["start"])
        if 'Labs' in combo.keys():
            if combo['Labs']['days'] in lec_days:
                if not integrity(combo['Lecture'], combo['Labs']):
                    return False
        elif 'Discs' in combo.keys():
            if combo['Discs']['days'] in lec_days:
                if not integrity(combo['Lecture'], combo['Discs']):
                    return False

        return True
    
    @staticmethod
    def compat_sections(section1, section2):
        return (section1["end"] < section2["start"]) or (section1["start"] > section2["end"])

    def compat_combo(self, combo):
        for key in combo:
            section = combo[key]
            days = section["days"]
            if days == "M":
                for mon_section in self.monday:
                    if not self.compat_sections(section, mon_section):
                        return False
            elif days == "MW":
                for mon_section in self.monday:
                    if not self.compat_sections(section, mon_section):
                        return False
                for wed_section in self.wednesday:
                    if not self.compat_sections(section, wed_section):
                        return False
            elif days == "MWF":
                for mon_section in self.monday:
                    if not self.compat_sections(section, mon_section):
                        return False
                for wed_section in self.wednesday:
                    if not self.compat_sections(section, wed_section):
                        return False
                for fri_section in self.friday:
                    if not self.compat_sections(section, fri_section):
                        return False
            elif days == "T":
                for tue_section in self.tuesday:
                    if not self.compat_sections(section, tue_section):
                        return False
            elif days == "TH":
                for tue_section in self.tuesday:
                    if not self.compat_sections(section, tue_section):
                        return False
                for thu_section in self.thursday:
                    if not self.compat_sections(section, thu_section):
                        return False
            elif days == "W":
                for wed_section in self.wednesday:
                    if not self.compat_sections(section, wed_section):
                        return False
            elif days == "H":
                for thu_section in self.thursday:
                    if not self.compat_sections(section, thu_section):
                        return False
            elif days == "F":
                for fri_section in self.friday:
                    if not self.compat_sections(section, fri_section):
                        return False
        return True

    def sort(self):
        monday = sorted(self.monday, key = lambda x: x["start"])
        self.monday = monday
        tuesday = sorted(self.tuesday, key = lambda x: x["start"])
        self.tuesday = tuesday
        wednesday = sorted(self.wednesday, key = lambda x: x["start"])
        self.wednesday = wednesday
        thursday = sorted(self.thursday, key = lambda x: x["start"])
        self.thursday = thursday
        friday = sorted(self.friday, key = lambda x: x["start"])
        self.friday = friday

    # Static method indicates you want this to be a class function but that it doesn't need access to any class attributes
    @staticmethod
    def section_output(message, section):
        if section["type"] == "Lec":
            message += "\n\t\tID: " + section["id"] + " " + section["start"].strftime('%I:%M%p') + "-" + section["end"].strftime('%I:%M%p') + " (" + section["code"] + ": " + section["type"] + ")" + " Instructor(s): " + str(section["instructor"])
        else:
            message += "\n\t\tID: " + section["id"] + " " + section["start"].strftime('%I:%M%p') + "-" + section["end"].strftime('%I:%M%p') + " (" + section["code"] + ": " + section["type"] + ")"
        return message

    # Format function for object
    def output(self):
        self.sort()
        message = ""
        message += "\n\tMonday:"
        for section in self.monday:
            message = self.section_output(message, section)
        message += "\n\tTuesday:"
        for section in self.tuesday:
            message = self.section_output(message, section)
        message += "\n\tWednesday:"
        for section in self.wednesday:
            message = self.section_output(message, section)
        message += "\n\tThursday:"
        for section in self.thursday:
            message = self.section_output(message, section)
        message += "\n\tFriday:"
        for section in self.friday:
            message = self.section_output(message, section)
        message += "\n"
        return message

In [0]:
#@title
from time import time as time_now
from sys import exit
from requests import session
from datetime import datetime as dt
from folium import Map, Marker, Icon, Popup

# Takes in course codes as strings
course_codes = input("Please list all courses you would like in your schedule as course codes separated by commas. Ex: BUAD-310,CSCI-104\n").upper()
# Stores course codes as list where each index corresponds to one course
course_codes = course_codes.split(",")
for i in range(len(course_codes)):
    while "-" not in course_codes[i]:
        course_codes[i] = input(course_codes[i] + " is not a valid course code, please provide a valid course code in the format DEPT-###: ").upper()
# Removes beginning and trailing whitespace from each course code
for i in range(len(course_codes)):
        course_codes[i].strip()       

# Contains integer section IDs for any section the user has already registered for
# Useful because the program otherwise will only be able to sample from classes with open spaces, but the user could already be registered for a full class
registrations = []
registered = input("Have you already registered for any courses? (y/n) ").upper()
while (registered != "Y") and (registered != "N"):
    registered = input("Please enter (y/n): ").upper()
if registered == "Y":
    registrations = input("Please list all section ids you have registered for separated by commas- including discussions, labs, and quizzes. Ex: 29917,29930,30188\n")
    # Stores section IDs as list where each index corresponds to one section
    registrations.split(",")
    # Removes beginning and trailing whitespace from each section ID
    for i in range(len(registrations)):
        registrations[i].strip()

# Set containing string form of names of professors to exclude from combinations
profs_to_exclude = set()
exclude_prof = input("Are there any professors you would like to exclude? (y/n) ").upper()
if exclude_prof == "Y":
    num_exclude = input("How many professors would you like to exclude? (Please enter numerical digits such as 1,2,3...): ")
    while not num_exclude.isdigit():
        num_exclude = input("Please enter numerical digits: ")
    num_exclude = int(num_exclude)
    for i in range(num_exclude):
        print("Professor", i + 1, end = " - ")
        profs_to_exclude.add(input("Please provide the name of the professor as 'firstname lastname': ").strip().title())

exclude_quiz = input("Would you like to exclude consideration of quiz sections? (y/n) ").upper()
while (exclude_quiz != "Y") and (exclude_quiz != "N"):
    exclude_quiz = input("Please enter (y/n): ").upper()

preferred_timings = {}
specific_times = input("Would you like to set specific times for each day to start and end? e.g the start of the day would be the start of your first class and the end of the day would the end of your last class, excluding quiz sections. \nPlease respond (y/n): ").upper()
while (registered != "Y") and (registered != "N"):
    registered = input("Please enter (y/n): ").upper()
if specific_times == "Y":
    raw_timings = {'monday': input("When would you like Monday to start and end? Please format as start,end in 24 hour format. ex: 09:00,15:00\n").split(","),
                   'tuesday': input("When would you like Tuesday to start and end? Please format as start,end in 24 hour format. ex: 09:00,15:00\n").split(","),
                   'wednesday': input("When would you like Wednesday to start and end? Please format as start,end in 24 hour format. ex: 09:00,15:00\n").split(","),
                   'thursday': input("When would you like Thursday to start and end? Please format as start,end in 24 hour format. ex: 09:00,15:00\n").split(","),
                   'friday': input("When would you like Friday to start and end? Please format as start,end in 24 hour format. ex: 09:00,15:00\n").split(",")}
    for key in raw_timings:
        preferred_timings[key]= {'start': dt.strptime(raw_timings[key][0].strip(), '%H:%M'), 'end': dt.strptime(raw_timings[key][1].strip(), '%H:%M')}

# Opens Requests session
s = session()
courses = []

# This function modifies the courses list by reference
class_search(course_codes, courses, s, exclude_quiz, registrations)

# Closes Requests session 
s.close()

# Generates all possible of combinations of arranging a course such that each combo contains one of each section type
# Section types are Lecture, Discussion, Quiz, Lab etc.
for course in courses:
    if course.organize(profs_to_exclude) == "NPC":
        print("There is no viable combination of lectures and supplementary sections for", course.code)
        exit()
    else:
        discussions = False
        labs = False
        quizzes = False
        course_keys = course.segments[0].keys()
        if "Discs" in course_keys:
            discussions = True
        if "Labs" in course_keys:
            labs = True
        if "Quizzes" in course_keys:
            quizzes = True

        for segment in course.segments:
            if len(segment.keys()) == 1:
                course.add_combo({"Lecture": segment["Lecture"]})
            elif discussions and labs:
                for disc in segment["Discs"]:
                    for lab in segment["Labs"]:
                        course.add_combo({"Lecture": segment["Lecture"], "Discs": disc, "Labs": lab})
            elif discussions:
                for disc in segment["Discs"]:
                    course.add_combo({"Lecture": segment["Lecture"], "Discs": disc})
            elif labs:
                for lab in segment["Labs"]:
                    course.add_combo({"Lecture": segment["Lecture"], "Labs": lab})
            if quizzes:
                for combo in course.combinations:
                    combo["Quizzes"] = segment["Quizzes"][0]

combinations = []
num_courses = len(courses)
# Using Tuples here ()
# Iterating through combinations of arranging a course and appends them to the list named combinations
for combo1 in courses[0].combinations:
    for combo2 in courses[1].combinations:
        if num_courses == 2:
            combinations.append((combo1, combo2))
        else:
            for combo3 in courses[2].combinations:
                if num_courses == 3:
                    combinations.append((combo1, combo2, combo3))
                else:
                    for combo4 in courses[3].combinations:
                        if num_courses == 4:
                            combinations.append((combo1, combo2, combo3, combo4))
                        else:
                            for combo5 in courses[4].combinations:
                                if num_courses == 5:
                                    combinations.append((combo1, combo2, combo3, combo4, combo5))
                                else:
                                    for combo6 in courses[5].combinations:
                                        if num_courses == 6:
                                            combinations.append((combo1, combo2, combo3, combo4, combo5, combo6))
                                        else:
                                            for combo7 in courses[6].combinations:
                                                if num_courses == 7:
                                                    combinations.append((combo1, combo2, combo3, combo4, combo5, combo6, combo7))

schedules = []
for combo in combinations:
    schedule = Schedule()
    for segment in combo:
        schedule.add_combo(preferred_timings, segment)
    # If the size of the combo is the number of courses desired
    if len(schedule.codes) == len(courses):
        schedules.append(schedule)

In [0]:
#@title
def sec_type(section_type):
    if section_type == "Lec":
        return "Lecture"
    elif section_type == "Disc":
        return "Discussion"
    elif section_type == "Lab":
        return "Lab"

# Returns dictionary of necessary information to form a map of a given day
def schedule_map(schedule_day):
    map_data = {}
    for section in schedule_day:
        # Excludes quiz sections
        if section['type'] != 'Quiz':
            info = '<center><font face = "verdana" size = "2"' + '<h3>' + section['code'] + '</h3></font><br>' + '<font face = "verdana" size = "2"' + sec_type(section['type']) + "<br>Day(s): " + section['days'] + "<br>" + section['start'].strftime('%I:%M%p') + "-" + section['end'].strftime('%I:%M%p') + "<br>(" + section['id'] + ")" + '</font></center>'
            # If building name already a key in the map_data
            if (section['location']['Abbreviation'] in map_data.keys()) and (info not in map_data[section['location']['Abbreviation']]['info']):
                map_data[section['location']['Abbreviation']]['info'] = map_data[section['location']['Abbreviation']]['info'] + info
            # If building name not yet a key in the map_data, adds it as a key along with value
            else:
                map_data[section['location']['Abbreviation']] = {'Coordinates': section['location']["Coordinates"], 'info': info}
    return map_data

In [0]:
#@title
def ScheduleMap (schedule_num, schedules):
    # Imports necessary libraries to create map of schedules
    from folium import FeatureGroup, LayerControl, TileLayer, PolyLine, Popup
    from folium.plugins import MeasureControl, AntPath
    from branca.element import IFrame

    # Contains Folium Map objects
    # Index corresponds to schedule number
    schedule = schedules[schedule_num - 1]
    # The location coordinates are centered at USC
    m = Map(location = [34.0219, 241.7147], width = '95%', tiles = None, zoom_start = 17, min_zoom = 16, max_zoom = 19)
    days = [['Monday', schedule.monday], ['Tuesday', schedule.tuesday], ['Wednesday', schedule.wednesday], ['Thursday', schedule.thursday], ['Friday', schedule.friday]]
    for day_name, day in days:
        # Creates a Folium FeatureGroup object to which you can add Popups for each day
        fg = FeatureGroup(name = day_name, overlay = False)
        map_data = schedule_map(day)
        # Set which tracks number of distinct locations there are classes in a given day
        path_locations = list()
        for key in map_data:
            # Some HTML to give the Popup message a cleaner look
            message = '<center><font face = "palatino"' +  '<h2><strong>' + key + '</strong></h2></font></center>' + map_data[key]['info']
            iframe = Popup(html = IFrame(html = message, width = 120, height = 120), max_width = 120, show = True)
            # Location parameter is a latitude and longitude accessed from dictionary of USC building abbreviations to their coordinates
            pop_up = Marker(location = map_data[key]['Coordinates'], popup = iframe, tooltip = '<center>' + key + '</center> ' + map_data[key]['info'], icon = Icon(color = 'orange', icon = 'info-sign'))
            # Adds popup to FeatureGroup
            pop_up.add_to(fg)
            # Adds coordinates of location to set tracking distinct locations
            path_locations.append(map_data[key]['Coordinates'])
        # If a given day has classes at more than one distinct location, create an AntPath
        if len(path_locations) > 1:
            # The delay parameter is just how slow the lines should move    
            AntPath(locations = path_locations, delay = 2500, color = "#990000").add_to(fg)
        fg.add_to(m)
    # Puts options guide to select day of schedule to view at the top right of the map
    LayerControl(collapsed = False, position = 'topright').add_to(m)
    # Specifies map style to use (CartoDB, OpenStreetMap, etc.)
    TileLayer('CartoDB Positron', overlay = True, control = False).add_to(m)
    # Adds guide to show distance as you zoom in or out
    MeasureControl(position = 'bottomleft', primary_length_unit = 'miles', secondary_length_unit = 'meters').add_to(m)
    # Appends Folium Map object to list of Map objects
    return m

In [0]:
#@title
# Iterates through schedule combinations and provides formatted output
print("\nPossible schedule combinations:", len(schedules), end = "\n\n")
for num, schedule in enumerate(schedules):
    print("Schedule ", num + 1)
    print(schedule.output(), end = "")

In [0]:
#@title
# Outputs map of any given combination
from IPython.display import display
view_map = input('Would you like to view an interactive map for any of the above schedules? (y/n) ').upper()
while (view_map != "N") and (view_map != "Y"):
  view_map = input("Please enter (y/n): ").upper()
while view_map == "Y":
    schedule_num = input("Which schedule would like to view as a map? Please provide the answer as a number, ex: 12\n")
    while not schedule_num.isdigit():
        schedule_num = input("Please enter numeric digits: ")
    display(ScheduleMap(int(schedule_num), schedules))
    view_map = input('Would you like to view another map for a different schedule? (y/n) ').upper()
    while (view_map != "N") and (view_map != "Y"):
        view_map = input("Please enter (y/n): ").upper()

print('Hope this helped!')